In [255]:
import numpy as np
from scipy.stats import uniform, norm, f, t
import statistics
from scipy import linalg
from sklearn.linear_model import LinearRegression
import random
from sklearn.model_selection import cross_validate

In [54]:
N = 50
x = np.zeros((6,50))
a = np.zeros(50)
for i in range(5):
    x[i,:] = uniform.rvs(loc=-1, scale=2, size = 50)
for j in range(50):
    a[j] = 2 + 3*x[0,j] - 2*x[1,j] + x[2,j] + x[3,j] - x[4,j]
    x[5,j] = norm.rvs(a[j], 1.5)

## a)

In [250]:
R_sq = np.zeros(6)
for i in range(6):
    y = x[i, :]
    TSS = np.sum(np.square(x[i] - np.mean(y)))
    
    psi = np.ones((50,6))
    psi[:,1:] = np.delete(x,i,0).T
    F_inv = np.linalg.inv(psi.T@psi)
    beta = F_inv@psi.T@y
    RSS = (y - psi@beta).T @ (y - psi@beta)
    
    R_sq[i] = 1 - RSS/TSS
R_sq

array([0.65716136, 0.43907875, 0.18315037, 0.08699621, 0.13264666,
       0.7254276 ])

=> η связан с ξ, ξ[k] не являются мультиколлинеарными

## b)

In [251]:
# "вручную" (β[0] - у нас последний коэффициент)
target_eta = np.array([x[5,0], x[5,12], x[5,19], x[5,25], x[5,38], x[5,49]])
target_ksi = np.ones((6,6))
for i in range(5):
    target_ksi[i, :] = np.array([x[i,0], x[i,12], x[5,19], x[i,25], x[i,38], x[i,49]])
betas_hand = linalg.solve(target_ksi, target_eta)
betas_hand

array([ -2.18578094,  -2.52272452,  18.5127631 ,   2.98558084,
         2.15250863, -12.51498916])

In [252]:
# с помощью встроенной библиотеки для линейной регресси, без нулевого веса (всегда можем убрать его)
x_reg = x[:5,:].T
y_reg = x[5,:]
reg = LinearRegression().fit(x_reg, y_reg)
betas = reg.coef_
betas

array([ 3.53455031, -2.30265472,  1.03475013,  0.62032976, -0.882746  ])

In [253]:
# проверяем значимость бутстрапом (можно было не бутстрапом, а через t(N-6))
delta_boot = np.zeros((1000,5))
y = np.zeros((6,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x[0, k] == y[0, m]:
                y[:, m] = x[:, k]
                break
        
    x_boot = y[:5,:].T
    y_boot = y[5,:]
    reg_boot = LinearRegression().fit(x_boot, y_boot)
    betas_boot = reg_boot.coef_
    delta_boot[i, :] = np.abs(betas_boot - betas)

In [254]:
for i in range(5):
    m = sum(np.sort(delta_boot[:, i])>=abs(betas[i]))
    p_value = m/1000
    print(p_value)
    if p_value > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
0.0
отвергаем гипотезу о незначимости коэффициента beta2
0.016
отвергаем гипотезу о незначимости коэффициента beta3
0.161
нет оснований отвергнуть гипотезу о незначимости коэффициента beta4
0.029
отвергаем гипотезу о незначимости коэффициента beta5


In [259]:
# сделаем не через бутстрап
for i in range(5):
    deltas = betas[i]/np.sqrt(RSS*F_inv[i,i])*(N-6)**0.5
    p_values = 2*(1 - t.cdf(abs(deltas), N-6))
    print(p_values)
    if p_values > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
9.57901842513209e-07
отвергаем гипотезу о незначимости коэффициента beta2
0.012553130449339811
отвергаем гипотезу о незначимости коэффициента beta3
0.09659604710288483
нет оснований отвергнуть гипотезу о незначимости коэффициента beta4
0.028715737846914502
отвергаем гипотезу о незначимости коэффициента beta5


## c)

In [100]:
R_eta = R_sq[5]
print('коэффициент детерминации: ', R_eta, sep='')

коэффициент детерминации: 0.7254276045479084


In [97]:
delta_est = R_eta**2/(1-R_eta**2) * (N - 6)/(6-1)
delta_est

9.775010058684916

Δ ~ F(6-1,50-1) = F(5,49)

P(Δ >= Δ_est|H0) ≈ 1.66*10^(-6) < 0.05 => отвергаем гипотезу о незначимости коэффициента детерминации

## d)

In [107]:
a_0 = 2
eta_0 = norm.rvs(a_0, 1.5)
print ('рандомное значение в нуле: ', eta_0, sep='')

рандомное значение в нуле: 1.0040616013905266


In [108]:
# построим численный доверительный интервал
delta_int = np.zeros(1000)
for i in range(1000):
    eta_boot = norm.rvs(a_0, 1.5)
    delta_int[i] = eta_boot - eta_0
print('доверительный интервал: (', np.sort(delta_int)[24], ', ', np.sort(delta_int)[974], ')', sep='')

доверительный интервал: (-1.9149373578256872, 3.943877652048334)


## e)

In [109]:
y = x[5,:]
e = np.zeros(50)
psi = np.ones((50,6))
psi[:,1:] = np.delete(x,5,0).T
F_inv = np.linalg.inv(psi.T@psi)
beta = F_inv@psi.T@y
e = y - psi@beta
e

array([ 0.84524464, -1.57755395,  2.09812418, -1.49202065,  1.54935236,
        1.25758238, -1.23354073,  0.30257575, -1.3615586 ,  1.38482371,
        0.18678295,  1.45290526,  3.54780015, -2.62454206, -0.73697774,
       -0.2119806 ,  1.39407606, -2.62690182,  1.39457922, -0.82394558,
        0.8141053 ,  2.36067592, -1.54258669, -2.01908733, -0.95516516,
       -2.87389487,  0.03970882,  1.04197525, -2.71415705,  2.43870054,
       -1.28864096,  0.4629264 , -1.30185075,  0.56268497, -0.83842327,
       -1.83163695, -0.60884156, -0.63329264, -1.18394076, -1.37980173,
        1.05438459,  1.74774322,  0.81145312,  0.37832657,  0.45807828,
        0.89220305,  2.78160038, -0.44180626,  0.1167645 ,  0.92697011])

In [121]:
I=0
for i in range(N):
    for j in range (i+1, N):
        if(e[i] > e[j]):
            I += 1
I

598

Δ ~> N(0,1)

In [124]:
delta_est = (I - N*(N-1)/4)/(N**3/36)**0.5
p_value = 2*(1 - norm.cdf(abs(delta_est)))
p_value

0.8056255996938093

=> нет оснований отвергнуть гипотезу о независимости ошибок измерения

## g)

In [137]:
eps = np.zeros(50)
eps = x[5,:] - reg.predict(x_reg)

In [140]:
sigma = np.median(np.abs(eps))/0.675
sigma

1.8452763741911282

In [141]:
quantity = 0
for i in range(50):
    if eps[i]<=-2*sigma or eps[i]>=2*sigma:
        quantity+=1
        print(i, ' является выбросом', sep='')
if quantity == 0:
    print('выбросов нет!')

выбросов нет!


## f)

In [246]:
#Колмогоров

def F_est(x):
    F = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        F[i] = np.argsort(x)[i]/x.shape[0]
    return F

In [245]:
teta2_est = (np.sum(np.square(eps))/N)**0.5
print('оценка дисперсии: ', teta2_est, sep = '')

оценка дисперсии: 1.5227623144342968


In [247]:
delta_eps = N**0.5 * np.max(np.abs(F_est(eps) - norm.cdf(eps, 0, teta2_est)))
delta_eps

6.119871757431944

In [249]:
delta_boot = np.zeros(10000)
for i in range(10000):
    y = np.array(random.sample((eps).tolist(), N, counts = [100 for i in range(N)]))
    teta2_boot = (np.sum(np.square(y))/N)**0.5
    F = norm(0, teta2_boot)
    delta_boot[i] = N**0.5 * np.max(abs(F_est(eps) - F.cdf(eps)))
k = sum(delta_boot < delta_eps)+1
p_value = 1 - (k-1)/10000
p_value

0.5139

=> нет оснований отвергнуть гипотезу о нормальности распределения ошибок

## h)

In [176]:
cv = np.zeros(50)
for i in range(50):
    x_test = x[:5,i].reshape((5,1)).T
    y_test = x[5,i]
    x_train = np.delete(x[:5,:].T, i, 0)
    y_train = np.delete(x[5,:], i)
    reg_cv = LinearRegression().fit(x_train, y_train)
    cv[i] = (reg_cv.predict(x_test) - y_test)**2
cv

array([9.31277254e-01, 3.36810531e+00, 5.34143147e+00, 3.15963605e+00,
       2.80886589e+00, 2.23925061e+00, 2.14578118e+00, 1.26545259e-01,
       2.31589483e+00, 2.24269782e+00, 4.05422992e-02, 2.78459723e+00,
       1.68268321e+01, 9.62319453e+00, 6.49292852e-01, 4.85498616e-02,
       3.01215968e+00, 9.79491548e+00, 2.37851706e+00, 1.06336668e+00,
       8.05006596e-01, 7.83852011e+00, 2.92051142e+00, 5.64509378e+00,
       1.16038567e+00, 1.09744588e+01, 1.99505058e-03, 1.61853389e+00,
       8.95701665e+00, 6.80763394e+00, 2.03498671e+00, 2.84381742e-01,
       2.24382557e+00, 3.84407851e-01, 8.42265281e-01, 4.56975746e+00,
       5.29240025e-01, 5.91062511e-01, 1.70772627e+00, 2.68190931e+00,
       1.26962113e+00, 3.65099576e+00, 7.31319856e-01, 1.63977966e-01,
       2.79975922e-01, 9.62570075e-01, 9.76874304e+00, 2.70515598e-01,
       1.67542268e-02, 1.26566732e+00])

In [177]:
CVSS = np.sum(cv)
TSS = np.sum(np.square(x[5,:] - np.mean(x[5,:])))
R_cv = 1 - CVSS/TSS
R_cv

0.6403135221517331

## i)

In [211]:
# пусть все ξ = 0.5
RSS = (x[5,:] - psi@beta).T @ (x[5,:] - psi@beta)
b = 2 + 0.5*(3-2+1+1-1)
eta_ad = np.zeros(5)
for i in range(5):
    eta_ad[i] = norm.rvs(b, 1.5)
sigma_cap = 1/4*np.sum(np.square(eta_ad - np.mean(eta_ad)))
delta_est = RSS/((N-6)*sigma_cap)
delta_est

2.4542290259088615

In [183]:
p_value = 1 - f.cdf(np.abs(delta_est), N-6, 4)
p_value

0.48827065834391803

=> нет оснований овергнуть гипотезу об адекватности модели

## j)

In [200]:
x_simpl = np.delete(x,3,0)

#b)
x_reg = x_simpl[:4,:].T
y_reg = x_simpl[4,:]
reg = LinearRegression().fit(x_reg, y_reg)
betas = reg.coef_

delta_boot = np.zeros((1000,4))
y = np.zeros((5,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x_simpl[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x_simpl[0, k] == y[0, m]:
                y[:, m] = x_simpl[:, k]
                break
        
    x_boot = y[:4,:].T
    y_boot = y[4,:]
    reg_boot = LinearRegression().fit(x_boot, y_boot)
    betas_boot = reg_boot.coef_
    delta_boot[i, :] = np.abs(betas_boot - betas)

for i in range(4):
    m = sum(np.sort(delta_boot[:, i])>=abs(betas[i]))
    p_value = m/1000
    print(p_value)
    if p_value > 0.05:
        print('нет оснований отвергнуть гипотезу о незначимости коэффициента beta', i+1, sep='')
    else:
        print('отвергаем гипотезу о незначимости коэффициента beta', i+1, sep='')

0.0
отвергаем гипотезу о незначимости коэффициента beta1
0.0
отвергаем гипотезу о незначимости коэффициента beta2
0.008
отвергаем гипотезу о незначимости коэффициента beta3
0.027
отвергаем гипотезу о незначимости коэффициента beta4


In [223]:
#c)
TSS = np.sum(np.square(x_simpl[4] - np.mean(x_simpl[4])))
    
psi = np.ones((50,5))
psi[:,1:] = np.delete(x_simpl,4,0).T
F_inv = np.linalg.inv(psi.T@psi)
beta = F_inv@psi.T@x_simpl[4]
RSS1 = (x_simpl[4] - psi@beta).T @ (x_simpl[4] - psi@beta)
    
R_sqrt = 1 - RSS1/TSS

delta_est = R_sqrt**2/(1-R_sqrt**2) * (N - 5)/(5-1)
p_value = 1 - f.cdf(np.abs(delta_est), 5-1, N-1)
print('коэффициент детерминации: ', R_sqrt, sep='')
print('p_value: ', p_value, sep='')

коэффициент детерминации: 0.7096656862660906
p_value: 1.2642254529904307e-06


=> отвергаем гипотезу о незначимости коэффициента детерминации

In [207]:
betas

array([ 3.63261237, -2.31209984,  0.98701639, -0.89699761])

коэффициенты ненамного отличаются от соответствующих коэффициентов до удаления ξ4

In [220]:
delta = (RSS1 - RSS)/RSS * (N-6)
p_value = 1 - f.cdf(delta,1,N-6)
p_value

0.11915616436609044

=> нет оснований отвергнуть гипотезу о том, что усложнение модели не является значимым

## k)

In [243]:
delta_boot = np.zeros(1000)
y = np.zeros((5,50))
for i in range(1000):
    y[0, :] = np.array(random.sample((x_simpl[0,:]).tolist(), N, counts = [100 for i in range(N)])) #хотим двумерную подвыборку или два цикла
    for m in range(50):
        for k in range(50):
            if x_simpl[0, k] == y[0, m]:
                y[:, m] = x_simpl[:, k]
                break
        
    TSS = np.sum(np.square(y[4] - np.mean(y[4])))
    
    psi = np.ones((50,5))
    psi[:,1:] = np.delete(y,4,0).T
    F_inv = np.linalg.inv(psi.T@psi)
    beta = F_inv@psi.T@y[4]
    RSS1 = (y[4] - psi@beta).T @ (y[4] - psi@beta)

    R_boot = 1 - RSS1/TSS
    delta_boot[i] = R_sq[5] - R_boot

delta_est = R_sq[5] - R_sqrt
m = sum(delta_boot >= delta_est)
p_value = m/1000
p_value

0.354

=> нет оснований отвергнуть гипотезу о том, что усложнение модели не является значимым